In [ ]:
# Import necessary libraries
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
# Function to detect face using OpenCV
def detect_face(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    
    if len(faces) == 0:
        return None, None
    
    (x, y, w, h) = faces[0]
    return gray[y:y+w, x:x+h], faces[0]



In [ ]:
# Function to prepare training data
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    
    for dir_name in dirs:
        if not dir_name.startswith("s"):
            continue
        label = int(dir_name.replace("s", ""))
        subject_dir_path = os.path.join(data_folder_path, dir_name)
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            if image_name.startswith("."):
                continue
            image_path = os.path.join(subject_dir_path, image_name)
            image = cv2.imread(image_path)
            face, rect = detect_face(image)
            
            if face is not None:
                faces.append(face)
                labels.append(label)
    
    return faces, labels


In [ ]:

# Prepare training data
print("Preparing data...")
faces, labels = prepare_training_data("training-data")



In [ ]:
# Initialize LBPH Face Recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()



In [ ]:
# Train the face recognizer
face_recognizer.train(faces, np.array(labels))
print("Data prepared and face recognizer trained successfully.")



In [ ]:
# Function to draw rectangle around face
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)



In [ ]:
# Function to draw text on image
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)




In [ ]:
# Function to predict faces in test images
def predict(test_img_path):
    img = cv2.imread(test_img_path)
    predicted_img = img.copy()
    face, rect = detect_face(predicted_img)
    label, confidence = face_recognizer.predict(face)
    label_text = subjects[label]
    
    draw_rectangle(predicted_img, rect)
    draw_text(predicted_img, label_text, rect[0], rect[1]-5)
    return predicted_img

